In [1]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)

# cropVideo(r"C:\Users\Dell\Downloads\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", r"media\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", 69,244,1676,942)
title = "Do you know! Ancient Egypt"
# title.replace(": ", " ")
downloadName = "TORz7ZRVEffb.. 🏛️ -D-1712844138"
recent_mp4 = os.path.join(os.path.expanduser("~/Downloads"), downloadName + ".mp4")

current_directory = os.path.join(os.getcwd(), "media")
path = os.path.join(current_directory, title + ".mp4")
cropVideo(recent_mp4, path, 69, 244, 1676, 942)
os.remove(recent_mp4)

Moviepy - Building video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Ancient Egypt.mp4.
MoviePy - Writing audio in Do you know! Ancient EgyptTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Ancient Egypt.mp4



Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Ancient Egypt.mp4
